# Take a set of calibrations
The script is built for the scriptQueue but for the moment should be run from the notebook

This script is used to take flats with the AuxTel calibration illumination system. <br>
It is being used as a stand-in until the proper functionality is build into the control system.

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import astropy

from lsst.ts import salobj
# from lsst.ts.externalscripts.auxtel.latiss_take_flats import LatissTakeFlats
from lsst.ts.externalscripts.auxtel import LatissTakeFlats

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Define funtion used to create the instrument setup for each flat that gets passed to the script

In [ ]:
def step_config(min_wave: float=500, max_wave: float=700, w_steps: int=10, grating: int = 1, 
                ent_slit: float = 4.5, exit_slit: float = 4.5, exp_time: float = 3,
                n_exp: int = 3, fs_exp_time: int = 3, fs_n_exp: int =1, em_exp_time: float=3, em_n_exp:int=1):
    waves = np.arange(min_wave,max_wave,w_steps) # r-band

    # config=dict(steps=[])
    steps = []
    
    spec_res = 999 # dummy value for now
    
    for wave in waves:
        step = {
    "wavelength": int(wave), # changed from float
    "grating": grating,  # enums.ATMonochromator.Grating.RED,  --> Enums are wrong!
    "spec_res": -1, # place holder until we know this
    "exit_slit_width": exit_slit,
    "entrance_slit_width": ent_slit,
    "exp_time": exp_time,
    "n_exp": n_exp,
    "fs_exp_time": fs_exp_time,
    "fs_n_exp": fs_n_exp,
    "em_exp_time": em_exp_time,
    "em_n_exp": em_n_exp}
    
        steps.append(step)
    
    # convert to yaml
    steps_for_script = yaml.safe_dump(steps)
    return steps_for_script

## Declare LATISS filter/grating combination

In [ ]:
latiss_filter = 'SDSSg_65mm' #'g'  # must be, g, r, i
latiss_grating= 'empty_1'

In [ ]:
if latiss_filter =='SDSSg_65mm':
    min_wave = 420; max_wave = 530  # g-band
elif latiss_filter == 'r':
    min_wave = 550; max_wave = 690  # r-band
elif latiss_filter == 'i':
    min_wave = 700; max_wave = 890  # i-band
else:
    raise "Filter not supported"

In [ ]:
w_steps = 20  # steps over the wavelength range
n_exp = 1 # number of LATISS exposures
exp_time = 5 # exposure time for each exposure
# Fiber spectrograph and electrometer must currently use the same exposure time.
fs_exp_time = exp_time 
em_exp_time = exp_time
fs_n_exp = n_exp
em_n_exp = n_exp
# following step creates the sequence
steps_for_config = step_config(min_wave=min_wave, max_wave=max_wave, w_steps=w_steps, 
                               exp_time=exp_time, n_exp=n_exp, fs_exp_time=fs_exp_time,
                               fs_n_exp=fs_n_exp, em_exp_time=em_exp_time, em_n_exp=em_n_exp)


## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissTakeFlats(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
# !conda info envs

In [ ]:
script.log.level = logging.DEBUG

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"latiss_filter": latiss_filter, 
                                "latiss_grating": latiss_grating,
                                "sequence": steps_for_config,
                                })

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
# ENUMS ARE WRONG!
# from lsst.ts.idl import enums
# enums.ATMonochromator.Grating.RED

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Elana Test: Loop through filters, wavelengths, and gratings

Note: will loop through g, r, i

Should we also loop through illuminator gratings? And what is the appropriate exposure time?